In [1]:
import seaborn as sns
import numpy as np
from sklearn.manifold import TSNE
import pandas as pd
from dataset.util import *
%matplotlib inline

In [2]:
# bert-serving-start -model_dir data/uncased_L-24_H-1024_A-16 -num_worker=1 -max_seq_len=NONE -max_batch_size 64
from bert_serving.client import BertClient
bc = BertClient()

In [20]:
(conn, cur) = sql_connect()
articles = cur.execute('SELECT * FROM articles').fetchall()
conn.close()
len(articles), articles[0]

(2995, [0])

In [21]:
(conn, cur) = sql_connect()
companies = cur.execute('SELECT * FROM companies').fetchall()
conn.close()
len(companies), companies[0]

(81,
 (1,
  'AAPL',
  'Apple, Inc.',
  'Computers/Consumer Electronics',
  'Technology',
  'Apple, Inc. engages in the design, manufacture, and sale of smartphones, personal computers, tablets, wearables and accessories, and other variety of related services. It operates through the following geographical segments: Americas, Europe, Greater China, Japan, and Rest of Asia Pacific. The Americas segment includes North and South America. The Europe segment consists of European countries, as well as India, the Middle East, and Africa. The Greater China segment comprises of China, Hong Kong, and Taiwan. The Rest of Asia Pacific segment includes Australia and Asian countries. Its products and services include iPhone, Mac, iPad, AirPods, Apple TV, Apple Watch, Beats products, Apple Care, iCloud, digital content stores, streaming, and licensing services. The company was founded by Steven Paul Jobs, Ronald Gerald Wayne, and Stephen G. Wozniak on April 1, 1976 and is headquartered in Cupertino, C

In [22]:
sym_to_comp = {}
for c in companies:
    sym_to_comp[c[1]] = c

In [4]:
texts = [a[4] for a in articles]

In [5]:
X = bc.encode(texts)

C:\Users\Shriv\Anaconda3\envs\tf-gpu\lib\site-packages\bert_serving\client\__init__.py:290: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency
  warnings.warn('server does not put a restriction on "max_seq_len", '


In [6]:
X_embedded = TSNE(n_components=2).fit_transform(X)

In [23]:
df = pd.DataFrame({
    'x': X_embedded[:, 0], 'y': X_embedded[:, 1],
    'symb': [a[1] for a in articles],
    'sector': [sym_to_comp[a[1]][4] for a in articles]
})

In [56]:
import plotly.express as px
fig = px.scatter(df, x="x", y="y", color="sector", hover_data=['sector', 'symb'])
fig.show()